# High-Performance Python

## Objectives

- Describe basic components of a computer
- Describe basic components of an operating system (OS)
- List components of a process
- State difference between processes & threads
- List issues involved in parallelizing computation

## Multi-Processing vs. Multi-Threading

Q: What is the difference between *multi-processing* and *multi-threading*?
- Multi-threading (also known as concurrency) splits the work between different threads running on the same processor. 
- When one thread is blocked the processor works on the tasks for the next one.
- Multi-processing splits work across processes running on different processors or even different machines.
- Multi-threading works better if you need to exchange data between the threads. 
- Multi-processing works better if the different processes can work heads down without communicating very much.

### Pop Quiz

<details>
<summary>Q: I have to process a very large dataset and run it through a CPU-intensive algorithm. Should I use multi-processing or multi-threading to speed it up?</summary>
A: Multi-processing will produce a result faster. This is because it will be able to split the work across different processors or machines.
</details>

<details>
<summary>Q: I have a web scraping application that spends most of its time waiting for web servers to respond. Should I use multi-processing or multi-threading to speed it up?
</summary>
A: Multi-threading will produce a bigger payoff. This is because it will ensure that the CPU is fully utilized and does not waste time blocked on input.
</details>

### Analogies

Multi-Threading | Multi-Processing
---|---
Laundromat | Everyone has a washer-dryer
Uber or Carpool | Everyone has a car

## Multi-Threading

Q: How can I write a multi-threaded program that prints `"hello"` in different threads?

- Import `threading`

In [1]:
import threading

- Define print as function.

In [2]:
from time import sleep

def print_with_delay(d, x):
    sleep(d)
    print x

- Create threads for printing.

In [3]:
t1 = threading.Thread(target = lambda: print_with_delay(1, 'hello with delay 1'))
t2 = threading.Thread(target = lambda: print_with_delay(2, 'hello with delay 2'))
t3 = threading.Thread(target = lambda: print_with_delay(3, 'hello with delay 3'))

- Start the threads.

In [4]:
t1.start()
t2.start()
t3.start()

- Wait for threads to finish.

In [5]:
t1.join()
t2.join()
t3.join()

hello with delay 1
hello with delay 2
hello with delay 3


### Multi-Processing

Q: Calculate the word count of strings using multi-processing.

- Import `Pool`

In [6]:
from multiprocessing import Pool

- Define how to count words in a string.

In [7]:
def word_count(string):
    return len(string.split())

- Define counting words sequentially.

In [8]:
def sequential_word_count(strings):
    return sum([word_count(string) for string in strings])

- Define counting words in parallel.

In [9]:
def parallel_word_count(strings):
    pool = Pool(processes = 4)
    results = pool.map(word_count, strings)
    return sum(results)

In [10]:
x = range(10)
print x
print map(lambda x: x ** 2, x)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


- Create `word_count` version that saves result in thread object.

In [11]:
def thread_word_count(string):
    self = threading.current_thread()
    self.result = word_count(string)

- Define counting words using `Thread`.

In [12]:
def concurrent_word_count(strings):
    threads = []
    
    for string in strings:
        thread = threading.Thread(
            target = thread_word_count,
            args = (string,))
        threads.append(thread)
        
    for thread in threads:
        thread.start()
        
    for thread in threads:
        thread.join()
        
    results = []
    for thread in threads: results.append(thread.result)
    return sum(results)

Q: Time all 3 versions.

- Create a sample input.

In [13]:
strings = [
    'hello world',
    'this is another line',
    'this is yet another line'] * 100000

- Time each one

In [14]:
%time print sequential_word_count(strings)

1100000
CPU times: user 181 ms, sys: 11.3 ms, total: 192 ms
Wall time: 197 ms


In [15]:
%time print concurrent_word_count(strings)

1100000
CPU times: user 21.9 s, sys: 14.2 s, total: 36.2 s
Wall time: 29.1 s


In [16]:
%time print parallel_word_count(strings)

1100000
CPU times: user 95.8 ms, sys: 25.3 ms, total: 121 ms
Wall time: 152 ms


### Pop Quiz

<details>
<summary>Q: Between sequential, parallel, and concurrent, which one is the fastest? Which one is the slowest? Why?</summary>
1. Parallel is the fastest. Sequential is second.  Concurrent is the slowest.
<br/>
2. Concurrent and parallel have a higher setup overhead. This is not recovered for small problems.
<br/>
3. Use these only if your processing takes longer than the setup overhead.
</details>

### Cleaning Up Zombie Python Processes

Here is how to kill all the processes that `multiprocessing` will bring up in the background.

```sh
ps ux | grep ipykernel | grep -v grep | awk '{print $2}' | xargs kill -9
```